# SHARK QUEST
## New Kids On The Block



# TODO

- Pickle ?
- Column names?

## Selecting
- Select relevant columns
- Analyse relevant columns

## Cleaning :
- Cast to appropriate data types
    - General cleaning
        - identify duplicates
            - fuzzy
        - removing duplicates
            - remove
            - merge
        - handling null values
            - remove
            - replace
        - manipulating strings
        - formatting the data.

- Wrong inputs
- Outliers

## Hypothesis

### TIME
- Shark attacks are seasonal (Summer)
- Shark attacks are increasing 
- Shark attacks are more concentrated in the PM 

### LOCATION
- Some countries are more attack prone (Australia)
- Some countries are more likely to be fatal (Australia)


### DEMOGRAPHICS
- Males are more likely to get attacked
- Males are more likely to get provoke a shark
- Provoked attacked are more fatal
- Young persons are more likely to get attacked
- Old persons are more likely to get killed

- Names more likely to get attacked (John)

### OTHER
- Some species are more aggressive (Tiger Shark)
- Some activities are more likely (Surfing)
- Some activities are more fatal

- Full moon? 😂


## Business Ideas

- App that gives a likelyhood of attack based on location and time
- Vacations far away from sharks for phobics
- Witness attacks for masochists

- Surf school at the safest places / seasons
- Fishing supplies => shark repellant by activities
- Safety training to avoid provocations / live in harmony

- Shark repellant => best spots
- Insurance for surfers, premiums for high risk areas

### GENERAL CLEANING
1. Rename columns
2. Drop columns
3. Remove duplicates
    - Remove full dupes
    - Remove fuzzy search
4. Strip values

### SPECIFIC CLEANING
1. Search / Replace / Reformat strings
2. Merge categories
3. Cast to Null
4. Cast correct type
5. Create new columns


In [ ]:
import pandas as pd
import re
import numpy as np


# Load the data
shark_attacks = pd.read_excel('GSAF5.xls')

shark_attacks[shark_attacks[['Case Number', 'Name']].duplicated(keep=False) & ~shark_attacks['Case Number'].isna()]
shark_attacks

In [115]:
# Settings
pd.set_option('display.max_rows', 80)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 100)

In [116]:
# renaming columns
shark_attacks.columns = [col.strip().replace(" ", "_").replace(
    ".", "").lower() for col in shark_attacks.columns]

In [ ]:
data_schema = {
    'type': {'dtype': 'category', 'categories': ['PROVOKED', 'UNPROVOKED', 'INVALID']},
    'date': {'dtype': 'datetime64[ns]'},
    'country': {'dtype': 'string'},
    'state': {'dtype': 'string'},
    'location': {'dtype': 'string'},
    'activity': {'dtype': 'category'},
    'severity': {'dtype': 'category', 'categories': ['FATALITY', 'INJURY', 'OTHER']},
    'injury': {'dtype': 'string'},
    'time': {'dtype': 'category', 'categories': ['MORNING', 'NOON', 'AFTERNOON', 'DANW', 'NIGHT', 'DUSK']}
}

business_relevant_columns = ['date', 'type', 'country',
                             'state', 'location', 'activity', 'injury', 'time']
display(business_relevant_columns)
shark_attacks[business_relevant_columns]

In [118]:
# dropping columns
# relevant_columns = [col for col in data_schema]
# Use boolean indexing to filter columns
# dropped_columns = shark_attacks.columns[~shark_attacks.columns.isin(relevant_columns)]
# display(dropped_columns)
# shark_attacks = shark_attacks.loc[:, relevant_columns]

In [119]:
#Remove duplicates
#Drop perfect duplicates
shark_attacks.drop_duplicates(keep=False, inplace=True)


# FUNCTIONS

In [ ]:
# Hennings functions
# Merge categories : (*categories to be merged, target) - Henning


def merge_values(row, arg1, *args):
    if row in args:
        return arg1
    else:
        return row


shark_attacks['type'].unique()

shark_attacks['type'] = shark_attacks['type'].apply(merge_values, args=(
    "Invalid", "Questionable", "Unconfirmed", "?", 'Unverified', 'Under investigation'))
shark_attacks['type'] = shark_attacks['type'].apply(
    merge_values, args=("Provoked", " Provoked"))
shark_attacks['type'] = shark_attacks['type'].apply(
    merge_values, args=("Watercraft", "Boat"))

shark_attacks['type'].unique()

In [ ]:
# Linh functions
# Strip function : strips spaces - Linh
def strip_func(df):
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: x.strip() if isinstance(x, str) else x)
    return df


df_cleaned = strip_func(shark_attacks)
df_cleaned.head(100)

In [ ]:
df_cleaned['location'] = df_cleaned['location'].astype(str)


def clean_data(location):
    location = re.sub(r'\d{1,2}º[NS],\s*\d{1,3}º[EW]', '', location)
    location = re.sub(r'\(.*?\)', '', location)
    location = re.sub(r'\s+', ' ', location)
    return location.title()


df_cleaned['location'] = df_cleaned['location'].apply(clean_data)
df_cleaned['location'].head(1000)

In [ ]:
categories = [
    (r"swimming|wading", "Swimming"),
    (r"surf|boogie boarding|paddle|foil", "Surfing"),
    (r"fish|fishing|spearfish|kayak", "Fishing"),
    (r"diving|scuba|freediving|snorkel", "Diving"),
    (r"sit|stand", "Passive"),
]


def categorize_activity(activity):
    if pd.isna(activity):
        return "Invalid"

    activity = activity.lower()

    for pattern, label in categories:
        if re.search(pattern, activity):
            return label

    return "Other Activity"


df_cleaned['activity'] = df_cleaned['activity'].apply(categorize_activity)
df_cleaned.head(100)

In [ ]:
# Ricardo functions
# Replace as Nan, Nat, ... function - Ricardo
# Remove obvious duplicate (entire line) - Ricardo
# Remove fuzzy duplicates (case number? dates?) - Ricardo

""" def repnan(schema, series, keys, value=nd.nan):

    # fills all

    # takes a list of values, replace with value OR <NaN> if no value is provided
    return series.replace(keys, value, inplace=True) """

def repnan(series, keys, value=np.nan):
    """
    input schema as data_schema for column types
    input series as dataframe['Column']
    input keys as a list of all the keys to replace
    input value as the replacer value for the keys, default is NaN
    """
    # fills all

    # takes a list of values, replace with value OR <NaN> if no value is provided
    return series.replace(keys, value, inplace=True)
    # changes series.type to match schema
    # output = series.astype(schema[series.name])

    #return output
    
repnan(shark_attacks["Name"].fillna(np.nan).str.strip(),['male', 'female'],np.nan)
shark_attacks

In [76]:
# Jp functions

# replacements
replacements_test_species = [
    (r'^.*([Tt]iger).*$', 'TIGER SHARK'),
]

replacements_test_dates = [
    (r'^(.*)(\d{2})(.*)(\w{3})(.*)(\d{4})(.*)', r'\2-\4-\6'),
    (r'^(.*)(\w{3})(.*)(\d{4})(.*)', r'01-\2-\4'),
    (r'-(uly|une)', r'-J\1'),
]

replacements_test_time = [
    (r'^.*((0[0-2]|23)h|([nN]ight|[dD]ark)).*$', 'NIGHT'),
    (r'^.*((0[3-6])h|([dD]awn|[sS]unrise|[dD]aybreak)).*$', 'DAWN'),
    (r'^.*((0[7-9]|10)h.*$|([mM]orning|^[aA]\.?[mM])).*$', 'MORNING'),
    (r'^.*((1[1-4])h.*$|([nN]oon|[mM]idday|[lL]unch)).*$', 'NOON'),
    (r'^.*((1[5-8])h.*$|([aA]fternoon|^[pP]\.?[mM])).*$', 'AFTERNOON'),
    (r'^.*((19|2[0-2])h.*$|([dD]usk|[sS]unset|[eE]vening)).*$', 'DUSK'),
]

In [ ]:

def replace_string_patterns(value, replacements):

    if isinstance(value, str):

        for pattern, target in replacements:
            value = re.sub(pattern, target, value)
        return value

    else:

        return value


species_replace = shark_attacks['Species '].apply(
    replace_string_patterns, replacements=replacements_test_species)
display(species_replace)

In [ ]:
# DATE TEST
date_replace_test = shark_attacks['date'].apply(
    replace_string_patterns, replacements=replacements_test_dates,)
date_test = pd.to_datetime(date_replace_test, format='mixed', errors='coerce')

formatted_dates = date_test.dt.strftime('%d-%m-%Y')
formatted_dates = formatted_dates.astype('datetime64[ns]')
formatted_dates

In [ ]:
# TIME TEST
display(shark_attacks['time'].value_counts())
time_replace_test = shark_attacks['time'].apply(
    replace_string_patterns, replacements=replacements_test_time,)
display(time_replace_test.value_counts())
time_replace_test

In [ ]:
valid_categories = [value for key, value in replacements_test_time]
# PRINT ALL COLUMNS WITH FIRST VALUE
shark_attacks.iloc[0]

In [ ]:
# Source, PDF, Case Number as possible duplicate finder or year / date fill
relevant_columns = ['date', 'Year', 'type', 'country', 'state', 'location', 'activity',
                    'Name', 'Sex', 'Age', 'injury', 'time', 'Species', 'Source', 'pdf', 'href formula', 'href']
business_relevant_columns = ['date', 'type', 'country',
                             'state', 'location', 'activity', 'injury', 'time']

# Henning : Date, Type
# Ricardo : Country, State
# Linh : Location, Activity
# Jp : Injury, Time

# Type : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Provoked, Unprovoked, Invalid
# Date : datetime : Clean "Reported" - Harmonize Format - Cast weird into NaT. Final => Dates (as datetime), NaT
# Country : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# State : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# Location : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# Activity : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Few categories to be determined
# Injury : category : Merge columns based on keywords. Nan into other. Final => Fatality, Injury, Other
# Time : category : Cast into categories Final => morning (6-10) noon (10-14) afternoon( 14-18) dusk (18-22) night (22 - 2) dawn (2-6) maybe as integers (0-5)

display(time_replace_test.value_counts())
where_test = time_replace_test.where(time_replace_test.isin(valid_categories))
where_test = where_test.astype('category')
where_test

In [ ]:
# RENAME COLUMNS

In [ ]:
# RETYPE COLUMNS
columns_types = {key: value['dtype'] for key, value in data_schema.items()}
print(columns_types)
shark_attacks.astype(columns_types)

In [ ]:
shark_attacks.shape

In [ ]:
shark_attacks.head()

In [ ]:
# PRINT ALL COLUMNS WITH FIRST VALUE
shark_attacks.iloc[0]

In [69]:
# Source, PDF, Case Number as possible duplicate finder or year / date fill
relevant_columns = ['date', 'Year', 'type', 'country', 'state', 'location', 'activity',
                    'Name', 'Sex', 'Age', 'injury', 'time', 'Species', 'Source', 'pdf', 'href formula', 'href']
business_relevant_columns = ['date', 'type', 'country',
                             'state', 'location', 'activity', 'injury', 'time']

# Henning : Date, Type
# Ricardo : Country, State
# Linh : Location, Activity
# Jp : Injury, Time

# Type : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Provoked, Unprovoked, Invalid
# Date : datetime : Clean "Reported" - Harmonize Format - Cast weird into NaT. Final => Dates (as datetime), NaT
# Country : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# State : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# Location : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
# Activity : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Few categories to be determined
# Injury : category : Merge columns based on keywords. Nan into other. Final => Fatality, Injury, Other
# Time : category : Cast into categories Final => morning (6-10) noon (10-14) afternoon( 14-18) dusk (18-22) night (22 - 2) dawn (2-6) maybe as integers (0-5)

# Todo
# Remove obvious duplicate (entire line) - Ricardo
# Remove fuzzy duplicates (case number? dates?) - Ricardo

# Functions :
# Merge categories : (*categories to be merged, target) - Henning
# Strip function : strips spaces - Linh
# Replace as Nan, Nat, ... function - Ricardo
# DONE - Replace by keyword function - Jp
# DONE - Cast to dateTime function
# Matching function (find similarities, keyword based?)
# DONE - Reformat dates, strings
# Filter function

In [ ]:
matching_rows = shark_attacks[shark_attacks['injury'].str.contains(
    'FATAL', case=False, na=False)]

print(matching_rows)

In [ ]:
# PRINT MULTIPLE COLUMNS

display(shark_attacks[['Case Number', 'Case Number.1']].dropna())

In [ ]:
pattern = r"Unnamed"
unnamed_columns = shark_attacks[shark_attacks.filter(
    regex=pattern).columns].dropna(thresh=1)
display(unnamed_columns.head(100))

In [ ]:
# PRINT ALL MISFORMATED DATES (YEAR END \d{4})
shark_attacks[~shark_attacks['date'].str.contains(
    r'.*\d{4}$', regex=True, na=False)]

In [ ]:
shark_attacks['Name'].value_counts()


def get_fname(row):
    return str(row).split()[0]


shark_attacks['first_name'] = shark_attacks['Name'].apply(get_fname)

shark_attacks['first_name'].value_counts().head(50)

### INJURY

## Problems
- Some are non-human damages (boat, kayak, boards)
- Some are non injuries (touched shark)
- Some don't have details
- Fatalities flags are non consistent
- Some incident are post-mortem
- Some are not incidents ('Aboriginal rock carving depicts man being attacked by a shark')
- Nan

## Additional
- Some provoked details that could be useful if missing elsewhere

## Solutions
- 3 categories : 
    - Fatality
    - Injury
    - Other

- Keyword(s) based search / replace ?

### TIME

## Problems
- Lots of Nan
- Some are times (12h00,...), other are time of day (morning,...)
- Some are text
- Some are ranges
- Some are calculations
- Some are 12h based, other 24h based
- Some maybe duplicates 
- Some are misplaced data

## Solutions
- 2 types :
    - Time (int? str?)
    - Unknown / NaN / None
